# PSMC: Principles of the Spin Model Checker

convention:
- SMC: Gerard J. Holzmann. The Spin Model Checker: Primer and Reference Manual. Addison-Wesley, Reading, MA, 2004.
- MLCS: Mordechai Ben-Ari. Mathematical Logic for Computer Science (Second Edition). Springer, London, 2001.
- PCDP: M. Ben-Ari. Principles of Concurrent and Distributed Programming (Second Edition). Addison-Wesley, Harlow, UK, 2006.

| #   | Title                               | Progress | Description |
| :-- | :---------------------------------- | :------- | :---------- |
| 1   | Sequential Programming in PROMELA   | 100%     | 2022-12-21  |
| 2   | Verification of Sequential Programs | 100%     | 2022-12-22  |
| 3   | Concurrency                         | 100%     | 2022-12-26  |
| 4   | Synchronization                     | 100%     | 2022-12-27  |
| 5   | Verification with Temporal Logic    | 100%     | 2022-12-28  |
| 6   | Data and Program Structures         | 100%     | 2022-12-28  |
| 7   | Channels                            | 100%     | 2022-12-28  |
| 8   | Nondeterminism                      | 100%     | 2022-12-28  |
| 9   | Advanced Topics in PROMELA          | 100%     | 2022-12-29  |
| 10  | Advanced Topics in SPIN             | 100%     | 2022-12-29  |
| 11  | Case Studies                        | xxx%     | yyyy-mm-dd  |

In [ ]:
%cd /mnt/d/GoogleDrive/wiki/jupyter-notebooks/Modeling/SPIN/PSMC

/mnt/d/GoogleDrive/wiki/jupyter-notebooks/Modeling/SPIN/PSMC


#  1. Sequential Programming in PROMELA

In [ ]:
%cd /mnt/d/GoogleDrive/wiki/jupyter-notebooks/Modeling/SPIN/PSMC/ch01

/mnt/d/GoogleDrive/wiki/jupyter-notebooks/Modeling/SPIN/PSMC/ch01


SPIN是一个模型检测器(model checker): 一个验证物理系统, 特别是计算机系统的软件工具.

1. 编写描述系统行为的模型
2. 描述表达系统行为需求的正确性属性(correctness properties)
3. 运行模型检测器以检查模型是否保持正确性属性, 如果不保持, 提供反例(counterexample): 不满足正确性属性的计算(computation).

Listing 1.1. Reversing digits

构造: 
- 赋值语句
- 表达式
- 进程: active proctype
- 注释 /* */, //
- printf

模拟模式(simulation mode): SPIN编译和执行PROMELA程序.

- 限制模拟运行的步数: spin -uN
- 过滤输出: jSPIN MSC配置
- Promela输入: STDIN channel

In [ ]:
!spin reversing_digits.pml

      value = 123,reversed = 321
1 process created


数值数据类型: bit, bool, byte, short, int, unsigned
- 尽量使用较少位的类型, 以避免验证中状态数量爆炸.

没有: 字符类型, 字符串变量, 浮点数类型

没有: 显式类型转换
- 算术先隐式转换成int, 赋值时再隐式转换成变量的类型.

操作符


Table 1.2. Operators in Promela

```
Precedence Operator Associativity Name
14 () left parentheses
14 [ ] left array indexing
14 . left field selection
13 ! right logical negation
13 ~ right bitwise complementation
13 ++, -- right increment, decrement
12 *, /, % left multiplication, division, modulo
11 +, - left addition, subtraction
10 <<, >> left left and right bitwise shift
9 <, <=, >, >= left arithmetic relational operators
8 ==, != left equality, inequality
7 & left bitwise and
6 ^ left bitwise exclusive or
5 | left bitwise inclusive or
4 && left logical and
3 || left logical or
2 ( -> : ) right conditional expression
1 = right assignment
```

表达式
- PROMELA中表达式必须是无副作用的.
- PROMELA与C语言的差异
  - 赋值语句不是表达式
  - `++`, `--`在赋值语句中只能用作后缀操作符, 不能用在赋值语句右侧的表达式中
  - 没有前缀`++`, `--`

```promela
b++ // ok

a = b++ // error
```

局部变量: 
- 作用域是其声明的进程
- 所有局部变量声明隐式的移动到进程的开始处

符号名:
- `#define N 10`: 数值的符号
- `mtype`: 值的助记名
  - 使用`%e`格式描述符/`printm`输出, 出现在程序的trace中
  - 一个程序中只有唯一的一组名称


控制语句:
- guarded commands: 表达不确定性
- location counter: 记录下一个执行的指令地址的寄存器
- control point: 控制点, 指令的地址
- 分类:
  * sequence: 顺序, 使用`;`分隔
  * selection: 选择
  * repetition: 重复
  * jump 跳转
  * `unless`


选择语句:
- `if`: `:: <guard> -> <statements>`
 - `<guard>`和`<statements>`之间可能存在interleaving(交错)
 - `else`: 所有其他`<guard>`求值为false时才执行
 - 所有`<guard>`求值为false时, 进程阻塞直到某个`<guard>`求值为true.
 - `skip`: `<statements>`为空
- conditional expression: `max = (a > b -> a : b)`
  - 必须有`()`, 该赋值是原子语句 


重复语句:
- `do`: 与`if`类似
  - `break`L 结束循环

宏: 见6.3.2节

跳转语句:
- `goto`

#  2. Verification of Sequential Programs

In [ ]:
%cd /mnt/d/GoogleDrive/wiki/jupyter-notebooks/Modeling/SPIN/PSMC/ch02

/mnt/d/GoogleDrive/wiki/jupyter-notebooks/Modeling/SPIN/PSMC/ch02


- 程序的状态state: 变量的值, location counter
- 程序的计算computation:
- 程序的状态空间state space

断言assertion
- `assert`
- precondition, invariant, postcondition

SPIN验证步骤:
- 从PROMELA源码生成验证器源码(C语言): `spin -a xxx.pml`
- 编译验证器 `pan`
- 执行验证器, 生成报告(错误时生成踪迹文件)


`pan`的参数:
- `-e`: 创建所有错误的踪迹(trail)
- `-cN`: 在第N次错误时停止. `-c0`忽略所有错误不产生踪迹文件.

踪迹文件: SPIN的guided模拟模式运行以重构计算
- `spin -t xxx.pml`

SPIN显示模拟中数据:
- `-p`: statements, 进程执行的语句
- `-g`: globals, 全局变量的值
- `-l`: locals, 局部变量的值
- `-s`: send, channel中执行的发送指令
- `-r`: receive, channel中执行的接收指令

In [ ]:
!spin -a max-error.pml
# -w: disable warnning message
!gcc -w -o pan pan.c
!./pan

pan:1: assertion violated ( ((a>=b)) ? ((max==a)) : ((max==b)) ) (at depth 0)
pan: wrote max-error.pml.trail

(Spin Version 6.5.1 -- 20 December 2019)
	+ Partial Order Reduction

Full statespace search for:
	never claim         	- (none specified)
	assertion violations	+
	acceptance   cycles 	- (not selected)
	invalid end states	+

State-vector 24 byte, depth reached 2, errors: 1
        3 states, stored
        0 states, matched
        3 transitions (= stored+matched)
        0 atomic steps
hash conflicts:         0 (resolved)

Stats on memory usage (in Megabytes):
    0.000	equivalent memory usage for states (stored*(State-vector + overhead))
    0.292	actual memory usage for states
  128.000	memory used for hash table (-w24)
    0.534	memory used for DFS stack (-m10000)
  128.730	total actual memory usage



pan: elapsed time 0.01 seconds


In [ ]:
# 验证: 上面的3步合并为1步
# need to mute gcc warning
!spin -run max-error.pml

pan.c: In function ‘make_trail’:
pan.c:1891:33: warning: ‘%d’ directive writing between 1 and 10 bytes into a region of size between 1 and 512 []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wformat-overflow=-Wformat-overflow=]8;;]
 1891 |                 sprintf(fnm, "%s%d.%s",
      |                                 ^~
pan.c:1891:30: note: directive argument in the range [1, 2147483647]
 1891 |                 sprintf(fnm, "%s%d.%s",
      |                              ^~~~~~~~~
In file included from /usr/include/stdio.h:894,
                 from pan.c:7:
/usr/include/x86_64-linux-gnu/bits/stdio2.h:38:10: note: ‘__builtin___sprintf_chk’ output 3 or more bytes (assuming 523) into a destination of size 512
   38 |   return __builtin___sprintf_chk (__s, __USE_FORTIFY_LEVEL - 1,
      |          ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
   39 |                                   __glibc_objsize (__s), __fmt,
      |                                   ~

In [ ]:
# follow simulation trail
!spin -t max-error.pml

spin: max-error.pml:7, Error: assertion violated
spin: text of failed assertion: assert(( ((a>=b)) -> ((max==a)) : ((max==b)) ))
spin: trail ends after 1 steps
#processes: 1
  1:	proc  0 (P:1) max-error.pml:8 (state 8) <valid end state>
1 process created


In [ ]:
# 显示trail中更多信息
!spin -t -p -g -l max-error.pml

  1:	proc  0 (P:1) max-error.pml:5 (state 3)	[((b>=a))]
  1:	proc  0 (P:1) max-error.pml:5 (state 4)	[max = (b+1)]
		P(0):max = 6
spin: max-error.pml:7, Error: assertion violated
spin: text of failed assertion: assert(( ((a>=b)) -> ((max==a)) : ((max==b)) ))
  1:	proc  0 (P:1) max-error.pml:7 (state 7)	[assert(( ((a>=b)) -> ((max==a)) : ((max==b)) ))]
		P(0):max = 6
spin: trail ends after 1 steps
#processes: 1
  1:	proc  0 (P:1) max-error.pml:8 (state 8) <valid end state>
		P(0):max = 6
		P(0):b = 5
		P(0):a = 5
1 process created


In [ ]:
# 清理
!rm -f pan.* pan *.trail

In [ ]:
!spin -p -g -l max-error.pml

  0:	proc  - (:root:) creates proc  0 (P)
  1:	proc  0 (P:1) max-error.pml:5 (state 3)	[((b>=a))]
  2:	proc  0 (P:1) max-error.pml:5 (state 4)	[max = (b+1)]
		P(0):max = 6
  3:	proc  0 (P:1) max-error.pml:7 (state 6)	[.(goto)]
spin: max-error.pml:7, Error: assertion violated
spin: text of failed assertion: assert(( ((a>=b)) -> ((max==a)) : ((max==b)) ))
#processes: 1
  4:	proc  0 (P:1) max-error.pml:7 (state 7)
		P(0):max = 6
		P(0):b = 5
		P(0):a = 5
1 process created


#  3. Concurrency
#  4. Synchronization
#  5. Verification with Temporal Logic
#  6. Data and Program Structures
#  7. Channels
#  8. Nondeterminism
#  9. Advanced Topics in PROMELA
#  10. Advanced Topics in SPIN
#  11. Case Studies

# Supplementary Material on Spin Version 6

> supplementary material: https://extras.springer.com/?query=978-1-84628-769-5

- the `for`-statement.
- the `select`-statement.
- `inline` definitions now create a new scope for variable declarations.
- the LTL formula can be written within the PROMELA program.

```promela
ltl mutex { []!(csp && csq) }
ltl nostarvation { []<>csp }

// no longer necessary to define symbols
ltl { [](critical <= 1) }
// a remote reference is considered an expression
ltl { []!(P@cs && Q@cs) }
// temporal operators can be given as keywords
ltl mutex { always !(csp && csq) }
ltl nostarvation { always eventually csp }
```

```shell
pan -N mutex
```